# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Web-Services-Management-Agent-(WSMA)" data-toc-modified-id="Web-Services-Management-Agent-(WSMA)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Web Services Management Agent (WSMA)</a></div><div class="lev2 toc-item"><a href="#Device-Configuration" data-toc-modified-id="Device-Configuration-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Device Configuration</a></div><div class="lev2 toc-item"><a href="#Setup-and-Preparation" data-toc-modified-id="Setup-and-Preparation-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Setup and Preparation</a></div><div class="lev2 toc-item"><a href="#Getting-started-with-WSMA" data-toc-modified-id="Getting-started-with-WSMA-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Getting started with WSMA</a></div><div class="lev2 toc-item"><a href="#Setting-up-a-global-WSMA-Object" data-toc-modified-id="Setting-up-a-global-WSMA-Object-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Setting up a global WSMA Object</a></div><div class="lev1 toc-item"><a href="#Using-WSMA-to-set-up-NETCONF" data-toc-modified-id="Using-WSMA-to-set-up-NETCONF-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Using WSMA to set up NETCONF</a></div><div class="lev2 toc-item"><a href="#Using-Format-Specification-for-Structured-Data" data-toc-modified-id="Using-Format-Specification-for-Structured-Data-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Using Format Specification for Structured Data</a></div><div class="lev1 toc-item"><a href="#More-Examples-to-play-with" data-toc-modified-id="More-Examples-to-play-with-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>More Examples to play with</a></div><div class="lev1 toc-item"><a href="#Make-Configuration-Persistent" data-toc-modified-id="Make-Configuration-Persistent-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Make Configuration Persistent</a></div><div class="lev1 toc-item"><a href="#Module-Documentation" data-toc-modified-id="Module-Documentation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Module Documentation</a></div>

# Web Services Management Agent (WSMA)
The Web Services Management Agent (WSMA) defines a mechanism through which a network device can be managed, configuration data information can be retrieved, and new configuration data can be uploaded and manipulated. 

WSMA uses Extensible Markup Language (XML)-based data encoding, that is transported by the Simple Object Access Protocol (SOAP), for the configuration data and protocol messages.

In this Jupyter Notebook we're going through some basic examples how to use the a simple Python module that provides an 'easy-to-use' abstraction layer for the WSMA service. It can be used for exec commands (like `show version` or `show ip interface brief`) or for configuration commands.

For additional information see the following links:
- [WSMA Configuration Guide, Cisco IOS Release 15.1M](http://www.cisco.com/c/en/us/td/docs/ios/netmgmt/configuration/guide/Convert/WSMA/nm_cfg_wsma.html)
- [Cisco IOS Web Services Management Agent Command Reference](http://www.cisco.com/c/en/us/td/docs/ios-xml/ios/wsma/command/wsma-cr-book/wsma-cr-a1.html)
- [WSMA Python Library](https://github.com/rschmied/wsma_python)


## Device Configuration
The network device needs to be configured for WSMA in the same way as it needs configuration to support Telnet, SSH, NETCONF or RESTCONF, to name a few device management options.

In its simplest form, put the following configuration onto the device:

```
ip http server
ip http authentication local
wsma agent exec
 profile WSMA
wsma agent config
 profile WSMA
wsma profile listener WSMA
 transport http
end
```

We have enabled the web server on the device, defined local authentication so locally defined users with the correct privileges can use the service and selected the HTTP transport. This is exactly how `rtr1` has been setup.

<p class="my-notify-info">The above configuration snippet supports WSMA over HTTP. Note that in a production environment you should use an *encrypted* transport like HTTPS or SSH.</p>


## Setup and Preparation
To start the demonstration, we need to setup some logging. We set the log level to `WARN` which is the default. 

Setting the level to `INFO` or even `DEBUG` produces considerable more logging output.

In [ ]:
import logging

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.WARN)

Please specify the device parameters below:

In [ ]:
USER = 'vagrant'
PASS = 'vagrant'
HOST = '172.20.20.10'

Note that the other router (`rtr-2`) is on `172.20.20.20`. Both routers have an additional user configured with username and password `cisco` and a privilege level of "1". Use this *non-privileged* user to see how it behaves when using unauthorized commands (like config commands with the "cisco" user).

## Getting started with WSMA
Now that we have set up logging, we can actually import the library and run some code. The first examples uses a `with/as` construct that automatically opens and tests the connection for us. If the test is working, we will get the WSMA object as `w` and can work with it.

When the code leaves the `with` statement, the connection to the device is automatically closed. This is in particular useful when using the SSH transport as in this case the session will only be established once (in the beginning) and not for every configuration or show command.

In the case of HTTP/S it *could* also be useful. However, the current implementation of the service seems to close the session after every request. 

In [ ]:
# import the library
import wsma

# use the with / as construct to create a WSMA object and work with it
with wsma.HTTP(HOST, USER, PASS, port=80, tls=False) as w:
    if w is not None:
        w.execCLI('show ip interface brief')
        print(w.output)

## Setting up a global WSMA Object
In the next example, for the purpose of this demonstration, we create a WSMA object and manually `connect()` it. This way, we can use it until the end of this Jupyter Notebook.

In [ ]:
w = wsma.HTTP(HOST, USER, PASS, port=80, tls=False)
# w = wsma.SSH(HOST, USER, PASS)
w.connect()
w._ping()
w.output

In [ ]:
print(w.hasSession)

<p class="my-notify-info">Try using SSH as a transport by commenting out the HTTP line and remove the comment from the SSH line. Does it work? Why not? Use `rtr2` with SSH. Does that work? Why?</p>


# Using WSMA to set up NETCONF
We're using WSMA to finalize the setup of the NETCONF agent. The way of provisioning the configuration into a Vagrant VM / router leaves the trustpoint without a valid key. We therefore need to

- delete the existing trustpoints
- unconfigure the NETCONF agent
- configure the NETCONF agent

When the NETCONF agent is again configured it creates the required keys (but only if they do not exist in the first place).

This is a very specific Vagrant configuration provisioning related issue and should not be an issue in the real world. But it illustrates a nice WSMA use-case.

First, let's delete the key material. This is not really necessary but results in a *clean slate*.

In [ ]:
w.config("crypto key zeroize")

In [ ]:
import re

w.execCLI("show crypto pki trustpoint")
tp_re = '^Trustpoint (TP-self-signed-[0-9]+)'
trustpoints = w.output
print(trustpoints)
changed = False

for tp in re.findall(tp_re, trustpoints):
    w.config("no crypto pki trustpoint %s" % tp)
    changed = True

if changed:
    w.config("no netconf-yang")
    w.config("netconf-yang")
    # w.execCLI("copy run start")
    # w.configPersist()

In [ ]:
m = re.search(tp_re, trustpoints)
if m is not None:
    w.execCLI("show crypto key mypubkey rsa %s" % m.group(1))
    print(w.output)

## Using Format Specification for Structured Data

In [ ]:
w.execCLI('show ip interface brief', format_spec='built-in')

In [ ]:
import json
print(json.dumps(w.odmFormatResult, indent=2))

OK, so how did this work? Even though that we're dealing with the CLI, the above command returned structured data! Well, unfortunately, this only works for a very few, select commands where a template / schema exists within IOS (hence the 'built-in' format specification).

It is possible to create your own specifications but that is kind of cumbersome and we're referring to more modern approaches to structured data and network device programability like NETCONF and RESTCONF.

For the time being, here's the command that shows you what schema templates do exist in the IOS device:

In [ ]:
w.execCLI('show format built-in')
print(w.output)

And, to give you some idea about the templates, here's the output for the format template we just used for the `show ip interface brief` command:

In [ ]:
w.execCLI('sh format built-in cli show ip interface brief')
print(w.output)

Or something that is a bit more complex. Essentially, by providing information for delimiters, length and position of data, containers and lists (*structured data*) can be created from programmatic screen scraping:

In [ ]:
w.execCLI('sh format built-in cli show inventory')
print(w.output)

The following code segment shows the device inventory of the virtual CSR. First, the original screen output from the CLI and then the output using the above specification:

In [ ]:
w.execCLI('show inventory')
print(w.output), 

print('\n%s\n' % ('#' * 40))

w.execCLI('show inventory', format_spec='built-in')
print(json.dumps(w.odmFormatResult, indent=2))

# More Examples to play with
Feel free to change the commands sent to the router to something else. Experiment with regular expressions and work with the resulting output of the commands.

Also be aware that you can get help for any Python command by using the `help()` command like shown in the box at the end of this notebook.

In [ ]:
w.execCLI('show cdp neighbor\nshow ip route summary\nshow ip ospf neighbor')
print(w.output)

In [ ]:
w.execCLI('show ip route | inc E1')
print(w.output)

# Make Configuration Persistent
Here's another method to save the configuration, e.g. make the configuration *persistent*: `configPersist()`. It is defined as 'copies the running configuration to the startup configuration so that it persists across reloads'.

In [ ]:
w.configPersist()

<p class="my-notify-info">It takes a few seconds to save the configuration as it would while doing the same via CLI. The *processing* is indicated by an asterisk left to the notebook cell (`[*]`).</p>

# Module Documentation
As with all Python code, documentation should be embedded and can be obtained by using the `help()` function. Try also `help(wsma.HTTP)` or `help(wsma.SSH)`.

In [ ]:
help(wsma)